In [1]:
from math import isclose
import random
import time
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def random_matrices(columns, rows, quantity=1):
    result = []
    for i in range(quantity):
        result.append(np.random.rand(rows, columns))
    return result

In [3]:
def backward_substitution(L, b):
    x = np.zeros_like(b, dtype=float)
    if L[L.shape[0]-1, L.shape[0]-1] == 0:
        raise ValueError('Układ równań nie ma rozwiązań.')
    
    for i in range(L.shape[0]-1, -1, -1):
        s = 0
        for j in range (i+1, min(L.shape)):
            s += L[i, j]*x[j]
        x[i] = (b[i] - s)/L[i, i]
    return x

In [4]:
A = np.array([[1, 0, 0], [0, 3, 0], [0, 0, 2]])
b = [6, 7, 4]
print('Znalezione rozwiązanie układu równań:', backward_substitution(A, b))
print('Wynik poprawny:', np.linalg.solve(A, b))

Znalezione rozwiązanie układu równań: [6.         2.33333333 2.        ]
Wynik poprawny: [6.         2.33333333 2.        ]


In [5]:
def forward_substitution(U, b):
    x = np.zeros_like(b, dtype=float)
    if U[0, 0] == 0:
        raise ValueError('Układ równań nie ma rozwiązań.')
    
    for i in range(len(U)):
        s = 0
        for j in range(i):
            s += U[i, j]*x[j]
        x[i] = (b[i]-s)/U[i, i]
    return x

In [6]:
A = np.array([[1, 0, 0], [-1, 3, 0], [0, -3, 3]])
b = [6, 2, -3]
print('Znalezione rozwiązanie układu równań:', forward_substitution(A, b))
print('Wynik poprawny:', np.linalg.solve(A, b))

Znalezione rozwiązanie układu równań: [6.         2.66666667 1.66666667]
Wynik poprawny: [6.         2.66666667 1.66666667]


In [7]:
def gaussian_elimination(A, b):
    n = A.shape[0]
    A_b = np.c_[A, b]
    for i in range(n):
        for k in range(i, n):
            if abs(A_b[k, i])>=1/(2**16):
                A_b[[k, i]] = A_b[[i, k]]
                break
        if k == n-1 and abs(A_b[k, i])<1/(2**16):
                break
        
        for j in range(i+1, n):
            const = -A_b[j, i]/A_b[i, i]
            A_b[j, i:] = A_b[j, i:] + const*A_b[i, i:]
    return A_b[:,:n], A_b[:, n]

In [8]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
b = np.array([1, 2, 3])
A = np.array([[1, 2, 0], [0, 5, 0], [0, 8, 0]])
b = np.array([2, 2, 3,])
U, c = gaussian_elimination(A, b)
print('Znaleziona postać zredukowana oraz wektor c:\n', 'U=', U, '\nc=', c)

Znaleziona postać zredukowana oraz wektor c:
 U= [[1 2 0]
 [0 5 0]
 [0 0 0]] 
c= [2 2 0]


In [9]:
def gaussian_det(A):
    n = A.shape[0]
    ilosc_translacji = 0
    for i in range(n):
        if abs(A[i, i])<1/(2**16):
            ilosc_translacji += 1
            for k in range(i, n):
                if abs(A[k, i])>=1/(2**16):
                    A[[k, i]] = A[[i, k]]
                    break
            if k == n-1 and abs(A[k, i])<1/(2**16):
                return 0
        
        for j in range(i+1, n):
            const = -A[j, i]/A[i, i]
            A[j, i:] = A[j, i:] + const*A[i, i:]
    det = 1
    for i in range(n):
        det = det*A[i, i]
    return (-1)**ilosc_translacji*det

In [10]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
#A = np.array([[1, 0, 1], [0, 1, 0], [0, 1, 0]])
print('Znaleziony wyznacznik:', gaussian_det(A))
print('Wynik poprawny:', np.linalg.det(A))

Znaleziony wyznacznik: -3
Wynik poprawny: -3.0000000000000004


In [11]:
n = 10000
i = 0
for array in random_matrices(4, 4, n):
    if isclose(gaussian_det(array), np.linalg.det(array)):
        i+=1
print('Otrzymano', i, 'poprawnych wyników na', n, 'testów.')

Otrzymano 10000 poprawnych wyników na 10000 testów.


In [12]:
def gaussian_inv(A):
    n = A.shape[0]
    ilosc_translacji = 0
    A_id = np.c_[A, np.identity(n)]
    for i in range(n):
        for k in range(i, n):
            if abs(A_id[k, i])>=1/(2**16):
                A_id[[k, i]] = A_id[[i, k]]
                pass
        if k == n-1 and abs(A[k, i])<1/(2**16):
            raise ValueError("Podana macierz jest osobliwa.")
        A_id[i, i:] = (1/A_id[i, i])*A_id[i, i:]
        
        for j in range(n):
            if i != j:
                const = -A_id[j, i]
                A_id[j, i:] = A_id[j, i:] + const*A_id[i, i:]
    return A_id[:, n:]

In [13]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
#A = np.array([[1, 0, 0], [0, 1, 0], [1, 1, 0]])
print('Znaleziona macierz odwrotna:\n', gaussian_inv(A))
print('Wynik poprawny:\n', np.linalg.inv(A))

Znaleziona macierz odwrotna:
 [[-0.66666667 -1.33333333  1.        ]
 [-0.66666667  3.66666667 -2.        ]
 [ 1.         -2.          1.        ]]
Wynik poprawny:
 [[-0.66666667 -1.33333333  1.        ]
 [-0.66666667  3.66666667 -2.        ]
 [ 1.         -2.          1.        ]]


In [14]:
#Spaghetti code oryginalny dla potomnych <3

def gaussian_rank_original(A):
    for i in range(A.shape[0]):
        for j in range(i, A.shape[1]):
            if np.any(A[i:, j]):
                A[:, [i, j]] = A[:, [j, i]]
                for k in range(i, A.shape[0]):
                    if abs(A[k, i])>=1/(2**16):
                        A[[k, i]] = A[[i, k]]
                        break
                break
            if j == A.shape[1]-1 and abs(A[i, j])<1/(2**16):
                return i
        
        for l in range(i+1, A.shape[0]):
            const = -A[l, i]/A[i, i]
            A[l, i:] = A[l, i:] + const*A[i, i:]
    return n

In [15]:
def gaussian_rank(A):
    for pivot in range(min(A.shape)):
        found_non_zero = False
        for col in range(pivot, A.shape[1]):
            for row in range(pivot, A.shape[0]):
                if abs(A[row, col])>=1/(2**16):
                    found_non_zero = True
                    A[:, [pivot, col]] = A[:, [col, pivot]]
                    A[[row, pivot]] = A[[pivot, row]]
                    break
            if found_non_zero:
                break
            if col==A.shape[1]-1 and not found_non_zero:
                return pivot
        
        for row in range(pivot+1, A.shape[0]):
            const = -A[row, pivot]/A[pivot, pivot]
            A[row, pivot:] = A[row, pivot:] + const*A[pivot, pivot:]
    return min(A.shape)

In [16]:
A = np.array([[0, 1, 1], [0, 0, 1]])
print('Znaleziony rząd:', gaussian_rank(A))
print('Wynik poprawny:', np.linalg.matrix_rank(A))

Znaleziony rząd: 2
Wynik poprawny: 2


In [17]:
n1 = 1000
n2 = 1000
n3 = 1000
i = 0
for array in random_matrices(3, 5, n1):
    if gaussian_rank(array)==np.linalg.matrix_rank(array):
        i+=1
for array in random_matrices(5, 3, n2):
    if gaussian_rank(array)==np.linalg.matrix_rank(array):
        i+=1
for array in random_matrices(4, 4, n3):
    if gaussian_rank(array)==np.linalg.matrix_rank(array):
        i+=1
print('Otrzymano', i, 'poprawnych wyników na', n1+n2+n3, 'testów.')

Otrzymano 3000 poprawnych wyników na 3000 testów.
